In [ ]:
from pyspark.sql.functions import col, isnan, isnull, when, count, udf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json

In [8]:
config_file_path = 'config.json'

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

#spark log level to show errors
sc.setLogLevel("ERROR")
#setting up path + reading my csv from bucket
bucket = config["bucket"]
filename = config["cleaningfilename"]
file_path = 'gs://'+ bucket+'/'+filename

sdf= spark.read.csv(file_path, sep=',', header=True, inferSchema=True)

sdf.printSchema()

root
 |-- legId: string (nullable = true)
 |-- searchDate: date (nullable = true)
 |-- flightDate: date (nullable = true)
 |-- startingAirport: string (nullable = true)
 |-- destinationAirport: string (nullable = true)
 |-- fareBasisCode: string (nullable = true)
 |-- travelDuration: string (nullable = true)
 |-- elapsedDays: integer (nullable = true)
 |-- isBasicEconomy: boolean (nullable = true)
 |-- isRefundable: boolean (nullable = true)
 |-- isNonStop: boolean (nullable = true)
 |-- baseFare: double (nullable = true)
 |-- totalFare: double (nullable = true)
 |-- seatsRemaining: integer (nullable = true)
 |-- totalTravelDistance: integer (nullable = true)
 |-- segmentsDepartureTimeEpochSeconds: string (nullable = true)
 |-- segmentsDepartureTimeRaw: string (nullable = true)
 |-- segmentsArrivalTimeEpochSeconds: string (nullable = true)
 |-- segmentsArrivalTimeRaw: string (nullable = true)
 |-- segmentsArrivalAirportCode: string (nullable = true)
 |-- segmentsDepartureAirportCode: s

In [9]:
sdf.select([count(when(isnull(c), c)).alias(c) for c in ['totalTravelDistance','segmentsEquipmentDescription', 'seatsRemaining', 'segmentsAirlineName', 'totalFare']]).show()
#rechecking null values

sdf.select([count(when(isnull(c), c)).alias(c) for c in ['legId', 'searchDate', 'flightDate', 'startingAirport', 'destinationAirport', 'fareBasisCode', 'travelDuration', 'elapsedDays', 'isBasicEconomy', 'isRefundable', 'isNonStop', 'baseFare', 'segmentsDepartureTimeEpochSeconds', 'segmentsDepartureTimeRaw', 'segmentsArrivalTimeEpochSeconds', 'segmentsArrivalTimeRaw', 'segmentsArrivalAirportCode', 'segmentsDepartureAirportCode', 'segmentsAirlineCode', 'segmentsDurationInSeconds', 'segmentsDistance', 'segmentsCabinCode'
]]).show()
#rechecking null values

sdf=sdf.na.drop(subset=['totalTravelDistance','segmentsEquipmentDescription'])
#dropping nan values

+-------------------+----------------------------+--------------+-------------------+---------+
|totalTravelDistance|segmentsEquipmentDescription|seatsRemaining|segmentsAirlineName|totalFare|
+-------------------+----------------------------+--------------+-------------------+---------+
|            6094532|                     1557592|             0|                  0|        0|
+-------------------+----------------------------+--------------+-------------------+---------+



+-----+----------+----------+---------------+------------------+-------------+--------------+-----------+--------------+------------+---------+--------+---------------------------------+------------------------+-------------------------------+----------------------+--------------------------+----------------------------+-------------------+-------------------------+----------------+-----------------+
|legId|searchDate|flightDate|startingAirport|destinationAirport|fareBasisCode|travelDuration|elapsedDays|isBasicEconomy|isRefundable|isNonStop|baseFare|segmentsDepartureTimeEpochSeconds|segmentsDepartureTimeRaw|segmentsArrivalTimeEpochSeconds|segmentsArrivalTimeRaw|segmentsArrivalAirportCode|segmentsDepartureAirportCode|segmentsAirlineCode|segmentsDurationInSeconds|segmentsDistance|segmentsCabinCode|
+-----+----------+----------+---------------+------------------+-------------+--------------+-----------+--------------+------------+---------+--------+---------------------------------+------

In [10]:

sdf.select([count(when(isnull(c), c)).alias(c) for c in ['totalTravelDistance','segmentsEquipmentDescription', 'seatsRemaining', 'segmentsAirlineName', 'totalFare']]).show()
#checking

sdf.count()

+-------------------+----------------------------+--------------+-------------------+---------+
|totalTravelDistance|segmentsEquipmentDescription|seatsRemaining|segmentsAirlineName|totalFare|
+-------------------+----------------------------+--------------+-------------------+---------+
|                  0|                           0|             0|                  0|        0|
+-------------------+----------------------------+--------------+-------------------+---------+



74754290

In [11]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

sdf2 = sdf.limit(1000) 
vector_col="corr_features"
col = ['elapsedDays','isBasicEconomy', 'isNonStop', 'baseFare', 'totalFare', 'seatsRemaining', 'totalTravelDistance']
assembler = VectorAssembler(inputCols=col, outputCol=vector_col, handleInvalid="skip")

sdf2_vector= assembler.transform(sdf2).select(vector_col)
sdf2_vector
#creating correlation matrix to check for correlation between features + label 
#with help from https://stackoverflow.com/questions/52214404/how-to-get-the-correlation-matrix-of-a-pyspark-data-frame
# and https://www.statology.org/pyspark-correlation-matrix/

matrix= Correlation.corr(sdf2_vector, vector_col).collect()[0][0]
matrix

corr_matrix = matrix.toArray().tolist() 
corr_matrix_df = pd.DataFrame(data=corr_matrix, columns = col, index=col) 
corr_matrix_df

#creating correlation matrix to check for correlation between features + label 
#with help from https://stackoverflow.com/questions/52214404/how-to-get-the-correlation-matrix-of-a-pyspark-data-frame
# and https://www.statology.org/pyspark-correlation-matrix/

,elapsedDays,isBasicEconomy,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance
elapsedDays,1.000000,0.291493,-0.304016,-0.014603,-0.003857,0.257221,0.308360
isBasicEconomy,0.291493,1.000000,-0.231853,-0.217101,-0.210712,0.488183,0.359460
isNonStop,-0.304016,-0.231853,1.000000,0.088985,0.069098,-0.111609,-0.535982
baseFare,-0.014603,-0.217101,0.088985,1.000000,0.999762,-0.188334,0.018126
totalFare,-0.003857,-0.210712,0.069098,0.999762,1.000000,-0.184501,0.030322
seatsRemaining,0.257221,0.488183,-0.111609,-0.188334,-0.184501,1.000000,0.359107
totalTravelDistance,0.308360,0.359460,-0.535982,0.018126,0.030322,0.359107,1.000000


In [12]:
columns_to_drop= ['segmentsEquipmentDescription', 'elapsedDays', 'baseFare','legId', 'segmentsDurationInSeconds', 'segmentsArrivalTimeEpochSeconds', 'segmentsDepartureTimeEpochSeconds', 'segmentsDistance', 'segmentsArrivalTimeRaw', 'segmentsDepartureTimeRaw', 'fareBasisCode', 'segmentsAirlineName']
sdf = sdf.drop(*columns_to_drop)
sdf.printSchema()
#dropping cols based on unique identifiers/date/dependent vairble(base fare)/corr outcome with regards to correlation with total fare

sdf.show()

sdf2 = sdf.limit(500) #wouldnt run without limiting
pdf = sdf2.toPandas()  
pdf.head()

sdf.printSchema()


sdf.count()

root
 |-- searchDate: date (nullable = true)
 |-- flightDate: date (nullable = true)
 |-- startingAirport: string (nullable = true)
 |-- destinationAirport: string (nullable = true)
 |-- travelDuration: string (nullable = true)
 |-- isBasicEconomy: boolean (nullable = true)
 |-- isRefundable: boolean (nullable = true)
 |-- isNonStop: boolean (nullable = true)
 |-- totalFare: double (nullable = true)
 |-- seatsRemaining: integer (nullable = true)
 |-- totalTravelDistance: integer (nullable = true)
 |-- segmentsArrivalAirportCode: string (nullable = true)
 |-- segmentsDepartureAirportCode: string (nullable = true)
 |-- segmentsAirlineCode: string (nullable = true)
 |-- segmentsCabinCode: string (nullable = true)

+----------+----------+---------------+------------------+--------------+--------------+------------+---------+---------+--------------+-------------------+--------------------------+----------------------------+-------------------+-----------------+
|searchDate|flightDate|start

74754290

In [14]:
pdf.head()

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineCode,segmentsCabinCode
0,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,False,True,248.6,9,947,BOS,ATL,DL,coach
1,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,248.6,4,947,BOS,ATL,DL,coach
2,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,248.6,9,947,BOS,ATL,DL,coach
3,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,False,True,248.6,8,947,BOS,ATL,DL,coach
4,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,False,True,248.6,9,947,BOS,ATL,DL,coach


In [ ]:
output= config["cleaningfilename"]

In [15]:
output_file_path= 'gs://' + bucket + '/' + filename + output
sdf.write.parquet(output_file_path)